# Learning a Binary Classifier

## Importing Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df

## Deep Learning Deep Shit

In [ ]:
from collections import Counter

import keras
import keras.backend as k
import numpy as np
import tensorflow as tf
from keras.layers import (GRU, LSTM, BatchNormalization, Bidirectional, Dense,
                          Dropout, Conv2D, MaxPooling2D, Flatten)
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import multi_gpu_model, to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
BATCH_SIZE = 1024  # batch size for the network
EPOCH_NUMBER = 1  # number of epochs to train
THRESHOLD = 5  # symbols appearing fewer times will be replaced by a placeholder

## Preparing Data

In [ ]:
unique_symbols = Counter()

for _, message in df['url'].iteritems():
    unique_symbols.update(message)
    
print("Unique symbols:", len(unique_symbols))


# Find symbols that appear fewer times than the threshold:

uncommon_symbols = list()

for symbol, count in unique_symbols.items():
    if count < THRESHOLD:
        uncommon_symbols.append(symbol)

print("Uncommon symbols:", len(uncommon_symbols))

In [ ]:
# Replace them with a placeholder:
DUMMY = uncommon_symbols[0]
tr_table = str.maketrans("".join(uncommon_symbols), DUMMY * len(uncommon_symbols))

df['url'] = df['url'].apply(lambda x: x.translate(tr_table))

# We will need the number of unique symbols further down when we will decide on the dimensionality of inputs.
num_unique_symbols = len(unique_symbols) - len(uncommon_symbols) + 1 

In [ ]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['url'])

In [ ]:
mat = tokenizer.texts_to_sequences(df['url'])
#mat = pad_sequences(mat)

In [ ]:
import seaborn as sns

In [ ]:
arr = np.array([len(i) for i in mat])

In [ ]:
sns.distplot(arr)

In [ ]:
len(arr)

In [ ]:
np.argwhere(arr<300)[:,0].shape

In [ ]:
mat = np.array(mat)

In [ ]:
mat = mat[np.argwhere(arr<300)[:,0]]

In [ ]:
max_length = max([len(i) for i in mat])

In [ ]:
mat.shape

In [ ]:
max_length

In [ ]:
images = []
for m in tqdm_notebook(mat[:100000]):
    img = np.zeros((num_unique_symbols, max_length,1))
    for idx, e in enumerate(m):
        img[e, idx] = 1
    images.append(img)

In [ ]:
images = np.array(images)

In [ ]:
y = df['label']
y = (y=="bad")
y = y[:100000]

In [ ]:
len(y)

In [ ]:
y.sum()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, y, stratify=y.values, test_size=0.10)

## Defining Model

In [ ]:
sequence_length = images.shape[1]

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=images.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(images[:90000], y[:90000], epochs=5, batch_size=128)

In [ ]:
moe

In [ ]:
y_train.sum()

In [ ]:
model.evaluate(images[-10000:], y[-10000:])

In [ ]:
y_test.shape, y_test.sum()

In [ ]:
parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
dg = DataGenerator(x_train, y_train, batch_size=128)

In [ ]:
model.fit_generator(dg, epochs=10, use_multiprocessing=True, workers=12)

In [ ]:
parallel_model.fit_generator(dg, epochs=10, use_multiprocessing=True, workers=6)

In [ ]:
x_train.shape

In [ ]:
BATCH_SIZE = 10240
for epoch in range(EPOCH_NUMBER):
    print("Epoch", epoch)
    for i in range(0, len(x_train), BATCH_SIZE):
        batch = x_train[i:i+BATCH_SIZE]
        batch = to_categorical(batch, num_classes=num_unique_symbols)
        y_batch = y_train.iloc[i:i+BATCH_SIZE]
        model.fit(batch, y_batch, batch_size=256)

In [ ]:
model.fit(x_train, y_train, validation_split=0.1, batch_size=32)